# Bài tập 2 : Bài toán cộng hai vector
**Thông tin sinh viên** :

Hoàng Minh Thanh (18424062)

Jupyter notebook (Online) : https://colab.research.google.com/drive/189f2vZtNKGjc5oc5T6dkOJOVClT4rnS1

Thực hiện chạy trên Google Colab

Cài đặt plugin chạy GPU trên Google Colab

In [3]:
%%bash
nvcc --version
rm -rf /content/*
pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3pabvnts
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=beb7af5bbc89ed87a4ca76286fc8d0fb98af50bc7cf0a15e431563e374f66e1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2h0qs5z/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3pabvnts


In [4]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


Tải github private repository chứa file in.pnm cần lấy và các file khác

Vì đây là private repository nên phải cấu hình clone bằng ssh

Tham khảo tại : https://towardsdatascience.com/using-git-with-colab-via-ssh-175e8f3751ec

In [5]:
import os
os.chdir("/content/")
from google.colab import files
uploaded = files.upload()
!ls

Saving ssh.tar.gz to ssh.tar.gz
ssh.tar.gz


In [6]:
%%bash
rm -rf /root/.ssh
mkdir /root/.ssh

tar xvzf ssh.tar.gz
cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
chmod 700 /root/.ssh

touch /root/.ssh/known_hosts
ssh-keyscan github.com >> /root/.ssh/known_hosts
chmod 644 /root/.ssh/known_hosts
chmod 400 /root/.ssh/id_rsa_thanh

ssh-colab/
ssh-colab/config
ssh-colab/id_rsa_thanh
ssh-colab/id_rsa_thanh.pub


# github.com:22 SSH-2.0-babeld-9389d978
# github.com:22 SSH-2.0-babeld-9389d978
# github.com:22 SSH-2.0-babeld-9389d978


In [7]:
%%bash
git config --global user.email "hmthanhgm@gmail.com"
git config --global user.name "hmthanh"

ssh-agent /bin/bash
git clone git@github.com:hmthanh/paralell_programming.git

Cloning into 'paralell_programming'...


Di chuyển các file cần thiết ra /content/

In [8]:
%%bash
mv /content/paralell_programming/Practical_1/* /content/
rm -rf /content/paralell_programming/

## Cài đặt chương trình



In [9]:
%%cu
#include <stdio.h>
#include <stdint.h>
#include <string.h>

using namespace std;

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(EXIT_FAILURE);                                                    \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

void readPnm(char * fileName, int &width, int &height, uchar3 * &pixels)
{
	FILE * f = fopen(fileName, "r");
	if (f == NULL)
	{
		printf("Cannot read %s\n", fileName);
		exit(EXIT_FAILURE);
	}

	char type[3];
	fscanf(f, "%s", type);
	
	if (strcmp(type, "P3") != 0) // In this exercise, we don't touch other types
	{
		fclose(f);
		printf("Cannot read %s\n", fileName); 
		exit(EXIT_FAILURE); 
	}

	fscanf(f, "%i", &width);
	fscanf(f, "%i", &height);
	
	int max_val;
	fscanf(f, "%i", &max_val);
	if (max_val > 255) // In this exercise, we assume 1 byte per value
	{
		fclose(f);
		printf("Cannot read %s\n", fileName); 
		exit(EXIT_FAILURE); 
	}

	pixels = (uchar3 *)malloc(width * height * sizeof(uchar3));
	for (int i = 0; i < width * height; i++)
		fscanf(f, "%hhu%hhu%hhu", &pixels[i].x, &pixels[i].y, &pixels[i].z);

	fclose(f);
}

void writePnm(uchar3 * pixels, int width, int height, char * fileName)
{
	FILE * f = fopen(fileName, "w");
	if (f == NULL)
	{
		printf("Cannot write %s\n", fileName);
		exit(EXIT_FAILURE);
	}	

	fprintf(f, "P3\n%i\n%i\n255\n", width, height); 

	for (int i = 0; i < width * height; i++)
		fprintf(f, "%hhu\n%hhu\n%hhu\n", pixels[i].x, pixels[i].y, pixels[i].z);
	
	fclose(f);
}

__global__ void blurImgKernel(uchar3 *inPixels, int width, int height, 
							float *filter, int filterWidth, 
							uchar3 *outPixels)
{
	// TODO
	int r = blockIdx.y * blockDim.y + threadIdx.y;
	int c = blockIdx.x * blockDim.x + threadIdx.x;
	if (r < height && c < width)
	{
		int i = r * width + c;
		float3 outPixel = make_float3(0, 0, 0);
		for (int fR = 0; fR < filterWidth; fR++)
		{
			for (int fC = 0; fC < filterWidth; fC++)
			{
				float filterVal = filter[fR * filterWidth + fC];

				int inPixelsR = (r - filterWidth/2) + fR;
				int inPixelsC = (c - filterWidth/2) + fC;
				inPixelsR = min(height - 1, max(0, inPixelsR)); 
				inPixelsC = min(width - 1, max(0, inPixelsC)); 
				uchar3 inPixel = inPixels[inPixelsR * width + inPixelsC];
				
				outPixel.x += filterVal * inPixel.x;
				outPixel.y += filterVal * inPixel.y;
				outPixel.z += filterVal * inPixel.z;
			}
		}

		outPixels[i] = make_uchar3(outPixel.x, outPixel.y, outPixel.z);
	}
}

void blurImg(uchar3 * inPixels, int width, int height, float * filter, int filterWidth, 
			uchar3 * outPixels,
			bool useDevice=false, dim3 blockSize=dim3(1, 1))
{
	GpuTimer timer;
	timer.Start();
	if (useDevice == false)
	{
		for (int outPixelsR = 0; outPixelsR < height; outPixelsR++)
		{
			for (int outPixelsC = 0; outPixelsC < width; outPixelsC++)
			{
				float3 outPixel = make_float3(0, 0, 0);
				for (int filterR = 0; filterR < filterWidth; filterR++)
				{
					for (int filterC = 0; filterC < filterWidth; filterC++)
					{
						float filterVal = filter[filterR * filterWidth + filterC];

						int inPixelsR = (outPixelsR - filterWidth/2) + filterR;
						int inPixelsC = (outPixelsC - filterWidth/2) + filterC;
						inPixelsR = min(height - 1, max(0, inPixelsR)); 
						inPixelsC = min(width - 1, max(0, inPixelsC)); 
	 					uchar3 inPixel = inPixels[inPixelsR * width + inPixelsC];
						
						outPixel.x += filterVal * inPixel.x;
						outPixel.y += filterVal * inPixel.y;
						outPixel.z += filterVal * inPixel.z;
					}
				}
				outPixels[outPixelsR * width + outPixelsC] = make_uchar3(outPixel.x, outPixel.y, outPixel.z);
			}
		}
	}
	else // Use device
	{
		// Chọn GPU thực thi câu lệnh    
		int dev = 0;
		cudaDeviceProp devProp;
		cudaGetDeviceProperties(&devProp, dev);
		CHECK(cudaSetDevice(dev));
		printf("GPU name: %s\n", devProp.name);
		printf("GPU compute capability: %d.%d\n", devProp.major, devProp.minor);

		// TODO
		// Allocate device memories
		uchar3 *d_inPixels, *d_outPixels;
		float *d_filter;
		CHECK(cudaMalloc(&d_inPixels, width * height * 3 * sizeof(uchar3)));
		CHECK(cudaMalloc(&d_outPixels, width * height * 3 * sizeof(uchar3)));
		CHECK(cudaMalloc(&d_filter, filterWidth * filterWidth * sizeof(float)));

		// Copy data to device memory
		CHECK(cudaMemcpy(d_inPixels, inPixels, width * height * 3 * sizeof(uchar3), cudaMemcpyHostToDevice));
		CHECK(cudaMemcpy(d_filter, filter, filterWidth * filterWidth * sizeof(float), cudaMemcpyHostToDevice));

		// Call kernel
		dim3 gridSize((width - 1) / blockSize.x + 1, (height - 1) / blockSize.y + 1);
		blurImgKernel<<<gridSize, blockSize>>>(d_inPixels, width, height, d_filter, filterWidth, d_outPixels);
		

		// Copy result from device memory
		CHECK(cudaMemcpy(outPixels, d_outPixels, width * height * sizeof(uchar3), cudaMemcpyDeviceToHost));

		// Free device memories
		CHECK(cudaFree(d_inPixels));
		CHECK(cudaFree(d_outPixels));
		CHECK(cudaFree(d_filter));
	}
	timer.Stop();
	float time = timer.Elapsed();
	printf("Processing time (%s): %f ms\n\n", 
    		useDevice == true? "use device" : "use host", time);
}

float computeError(uchar3 * a1, uchar3 * a2, int n)
{
	float err = 0;
	for (int i = 0; i < n; i++)
	{
		err += abs((int)a1[i].x - (int)a2[i].x);
		err += abs((int)a1[i].y - (int)a2[i].y);
		err += abs((int)a1[i].z - (int)a2[i].z);
	}
	err /= (n * 3);
	return err;
}

char * concatStr(const char * s1, const char * s2)
{
    char * result = (char *)malloc(strlen(s1) + strlen(s2) + 1);
    strcpy(result, s1);
    strcat(result, s2);
    return result;
}

// Vì chạy online trên Colab nên ta set sẵn các tham số
int main() //int argc, char ** argv
{
	int argc = 5;
	char argv[5][255] = { "./bt1", "/content/in.pnm", "/content/out.pnm", "32", "32" };
	
	if (argc !=3 && argc != 5)
	{
		printf("The number of arguments is invalid\n");
		return EXIT_FAILURE;
	}

	// Read input image file
	int width, height;
	uchar3 * inPixels;
	readPnm(argv[1], width, height, inPixels);
	printf("Image size (width x height): %i x %i\n\n", width, height);

	// Set up a simple filter with blurring effect
	int filterWidth = 9;
	float *filter = (float *)malloc(filterWidth * filterWidth * sizeof(float));
	for (int filterR = 0; filterR < filterWidth; filterR++)
	{
		for (int filterC = 0; filterC < filterWidth; filterC++)
		{
			filter[filterR * filterWidth + filterC] = 1. / (filterWidth * filterWidth);
		}
	}

	// Blur input image not using device
	uchar3 * correctOutPixels= (uchar3 *)malloc(width * height * sizeof(uchar3)); 
	blurImg(inPixels, width, height, filter, filterWidth, correctOutPixels);
	
    // Blur input image using device
	uchar3 * outPixels= (uchar3 *)malloc(width * height * sizeof(uchar3));
	dim3 blockSize(32, 32); // Default
	if (argc == 5)
	{
		blockSize.x = atoi(argv[3]);
		blockSize.y = atoi(argv[4]);
	}
	blurImg(inPixels, width, height, filter, filterWidth, outPixels, true, blockSize);

	// Compute mean absolute error between host result and device result
	float err = computeError(outPixels, correctOutPixels, width * height);
	printf("Error between device result and host result: %f\n", err);

	// Write results to files
	char *outFileNameBase = strtok(argv[2], "."); // Get rid of extension
	printf("outFileNameBase : %s", outFileNameBase);
	writePnm(correctOutPixels, width, height, concatStr(outFileNameBase, "_host.pnm"));
	writePnm(outPixels, width, height, concatStr(outFileNameBase, "_device.pnm"));

	// Free memories
	free(inPixels);
	free(outPixels);
	free(filter);
}

Image size (width x height): 512 x 512

Processing time (use host): 384.859680 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 2.104704 ms

Error between device result and host result: 0.000703
outFileNameBase : /content/out


# Thực thi với các kích thước filter khác nhau

* Biên dịch file "bt1.cu"

In [10]:
!nvcc bt1.cu -o bt1

In [13]:
!./bt1 in.pnm out.pnm 32 16

Image size (width x height): 512 x 512

Processing time (use host): 385.448364 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 2.096864 ms

Error between device result and host result: 0.000703


Như vậy có thể thấy kết quả nhỏ hơn 0.xxx

* Thực thi với kích thước filter 8x8

In [14]:
!./bt1 in.pnm out.pnm 8 8

Image size (width x height): 512 x 512

Processing time (use host): 394.060913 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 2.206176 ms

Error between device result and host result: 0.000703


* Thực thi với kích thước filter 16 x 16

In [15]:
!./bt1 in.pnm out.pnm 16 16

Image size (width x height): 512 x 512

Processing time (use host): 393.984985 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 2.061920 ms

Error between device result and host result: 0.000703


* Thực thi với kích thước filter 32 x 32

In [16]:
!./bt1 in.pnm out.pnm 32 32

Image size (width x height): 512 x 512

Processing time (use host): 385.749817 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 2.049952 ms

Error between device result and host result: 0.000703


* Thực thi với kích thước filter 64 x 64

In [17]:
!./bt1 in.pnm out.pnm 64 64

Image size (width x height): 512 x 512

Processing time (use host): 387.610596 ms

GPU name: Tesla P100-PCIE-16GB
GPU compute capability: 6.0
Processing time (use device): 1.925440 ms

Error between device result and host result: 124.033791


Có thể thấy độ lỗi lên đến **124.033791** khi thực thi với kích thước filter 64 x 64 đúng như với yêu cầu của thầy

### Giải thích :

Xảy ra lỗi như vậy là do các thread của mỗi block được phân vào các wrap (32 thread) khác nhau.
Trong GPU thì mỗi thread trong một wrap thực thi bất đồng bộ sử dụng chung một thanh nhớ register (bộ nhớ tạm trên cùng một wrap) nên có thể với kích thước filter lớn hơn 32 thì quá trình tính toán song song sẽ dấn đến một thread đã xử lý xong và cập nhật kết quả mới vào bộ nhớ toàn cục nhưng thread khác đang xử lý sẽ lấy kết quả pixels từ bộ nhớ tạm trên thanh ghi.